# Import libraries & dataframes

In [80]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import plotly.express as px
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp
import seaborn as sns

In [81]:
# import dataframes
path = r'C:\Users\muril\Data Analysis Projects\07-2023 Instacart Basket Analysis\02 Data'

In [3]:
ords = pd.read_csv(os.path.join(path,'Original Data','orders.csv'))

In [4]:
prods = pd.read_csv(os.path.join(path,'Original Data','products.csv'))

In [61]:
dept = pd.read_csv(os.path.join(path,'Original Data','departments.csv'))

In [82]:
cust = pd.read_csv(os.path.join(path,'Original Data','customers.csv'))

# 1. Orders

In [6]:
# check dimensions of orders
ords.shape

(3421083, 7)

## 1.1 Wrangling

In [7]:
# view first rows of dataframe
ords.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### 1.1.1 Datatypes

In [8]:
# check for (mixed) datatypes
ords.dtypes

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

order_id and user_id can be converted to strings, since they represent names rather than quantifiable data.

In [9]:
# convert order_id and user_id to strings
ords['order_id'] = ords['order_id'].astype(str)
ords['user_id'] = ords['user_id'].astype(str)

In [10]:
# check for (mixed) datatypes
ords.dtypes

order_id                   object
user_id                    object
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

### 1.1.2 Columns to drop

In [11]:
# investigate eval_set column
ords['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

On verification with the client, this column is not useful to the project and can be ignored, hence it will be dropped.

In [12]:
# drop eval_set
ords = ords.drop('eval_set', axis=1)

### 1.1.3 Columns to rename

In [13]:
# order_dow isn't intuitive and, on verification with the client, means order_day_of_week and so will be renamed
ords.rename(columns={'order_dow': 'order_day_of_week'}, inplace=True)

In [14]:
# review dataframe
ords.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [15]:
ords.dtypes

order_id                   object
user_id                    object
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

Wrangling steps on orders set complete.

## 1.2 Consistency checks

### 1.2.1 Duplicates

In [16]:
# check for duplicate values
ords.duplicated().sum()

0

There are no duplicate rows.

### 1.2.2 Missing values

In [17]:
# check for nulls
ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [18]:
# investigate days_since_prior_order column
ords[ords['days_since_prior_order'].isnull()]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,NaN
3420934,3189322,206206,1,3,18,NaN
3421002,2166133,206207,1,6,19,NaN
3421019,2227043,206208,1,1,15,NaN


In [19]:
# count the number of users
ords['user_id'].nunique()

206209

In [20]:
# find how many users have more than 1 null in the days_since_prior_order column
( ords.groupby('user_id')['days_since_prior_order'].apply(lambda x: x.isnull().sum()) >1 ).sum()

0

There is one null for every user in the days_since_prior_order column. Verification with the client confirms that this is because it is the first order made by the user. These nulls are not true missing values and will be left unchanged.

There are no missing values in the orders dataset.

### 1.2.3. Inaccuracies

In [21]:
# find summary statistics for the dataset
ords.describe()

,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


The counts are all correct for the dataset.

None of the summary statistics are suspicious, therefore we will proceed on the basis that are no errors in the dataset.

# 2. Products

In [22]:
# view dimensions of products dataset
prods.shape

(49693, 5)

In [23]:
# view dataset
prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


## 2.1 Wrangling

### 2.1.1 Datatypes

In [24]:
# check for mixed datatypes
prods.dtypes

product_id         int64
product_name      object
aisle_id           int64
department_id      int64
prices           float64
dtype: object

product_id, aisle_id and department_id can be converted to strings, since they represent names rather than quantifiable data.

In [25]:
# convert product_id, aisle_id and department_id to strings
prods['product_id'] = prods['product_id'].astype(str)
prods['aisle_id'] = prods['aisle_id'].astype(str)
prods['department_id'] = prods['department_id'].astype(str)

In [26]:
# check datatypes
prods.dtypes

product_id        object
product_name      object
aisle_id          object
department_id     object
prices           float64
dtype: object

There are no mixed datatypes to fix.

### 2.1.2 Columns to drop

There are no columns that need to be dropped.

### 2.1.3 Columns to rename

All column names are clear and succinct, no need to change any names.

## 2.2 Consistency checks

### 2.2.1 Duplicates

In [27]:
# check for duplicate values
prods.duplicated().sum()

5

In [28]:
# view duplicates
prods[prods.duplicated(keep='last')]

,product_id,product_name,aisle_id,department_id,prices
461,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18458,18458,Ranger IPA,27,5,9.2
26809,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35308,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35494,35491,Adore Forever Body Wash,127,11,9.9


In [29]:
# view counts of these rows
prods[prods['product_id'].isin([462,18458,26808,35306,35491])].value_counts()

Series([], dtype: int64)

Each of these rows are duplicated once. Drop them.

In [30]:
# drop duplicates
prods.drop_duplicates(inplace=True)

In [31]:
# check for duplicate values
prods.duplicated().sum()

0

In [32]:
# check new dimensions
prods.shape

(49688, 5)

Duplicates removed.

## 2.2.2 Missing values

In [33]:
# check for nulls
prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [34]:
# view rows with missing values
prods[prods['product_name'].isnull()]

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


All numerical data is intact and looks sensible. Since we have the product_id, request the information from the client to be added to the dataset and do not drop the rows, as this will unnecessarily distort the analysis.

## 2.2.3 Inaccuracies

In [35]:
# find summary statistics for the dataset
prods.describe()

,prices
count,49688.000000
mean,9.994254
std,453.542503
min,1.000000
25%,4.100000
50%,7.100000
75%,11.200000
max,99999.000000


Count returns correctly.

The standard deviation and max do not seem sensible. Let's investigate the column.

In [36]:
# find products with high prices
prods[prods['prices'] > 30]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


These are clearly erroneous.
We will replace them with the mean price of their department (without those values).

In [37]:
# remove prices for these products
prods.loc[prods['product_id'].isin(['21553', '33664']), 'prices'] = None

In [38]:
prods[prods['product_id'].isin(['21553','33664'])]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN
33666,33664,2 % Reduced Fat Milk,84,16,NaN


In [39]:
# take subset of department 16
dept16 = prods[prods['department_id'] == '16' ]
dept16

,product_id,product_name,aisle_id,department_id,prices
8,9,Light Strawberry Blueberry Yogurt,120,16,7.0
97,98,2% Yellow American Cheese,2,16,4.5
116,117,Petit Suisse Fruit,2,16,3.1
120,121,Sharp Cheddar,21,16,8.1
126,127,Marscapone,108,16,3.5
...,...,...,...,...,...
49621,49617,Creamy Mozzarella Sun-Dried Tomato & Basil Fla...,2,16,10.2
49626,49622,Original Powder Coffee Creamer,53,16,12.0
49632,49628,Yoghurt Blueberry,120,16,14.7
49648,49644,Feta Crumbles,2,16,6.5


In [40]:
# find mean of prices for department 16
dept16['prices'].mean()

8.016652161299682

In [41]:
# impute this value for the erroneous (now null) prices
prods.loc[prods['product_id'].isin(['21553', '33664']), 'prices'] = 8.016652161299682

In [42]:
# check imputation
prods[prods['product_id'].isin(['21553','33664'])]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,8.016652
33666,33664,2 % Reduced Fat Milk,84,16,8.016652


In [43]:
# find summary stats again
prods.describe()

,prices
count,49688.000000
mean,7.682167
std,4.200252
min,1.000000
25%,4.100000
50%,7.100000
75%,11.200000
max,25.000000


products dataset is now clean.

## 3. Departments

In [44]:
# check dimensions
dept.shape

(1, 22)

In [45]:
# view dataset
dept.head()

,department_id,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,department,frozen,other,bakery,produce,alcohol,international,beverages,pets,dry goods pasta,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing


Clearly the dataframe needs to be transposed.

### 3.1 Wrangling

### 3.1.1 Transpose dataframe


In [62]:
# transpose dept
dept = dept.T

In [63]:
# reset index column
dept.reset_index()

,index,0
0,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta


In [65]:
# extract column names
header = dept.iloc[0]

In [66]:
# take only the dataset with the headers
dept = dept[1:]

In [68]:
# add headers
dept.columns = header

In [69]:
dept

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


### 3.1.2 Datatypes

In [70]:
# check datatypes
dept.dtypes

department_id
department    object
dtype: object

### 3.1.3 Columns to drop

No columns need to be dropped.

### 3.1.4 Columns to rename

No columns need to be renamed.

## 3.2 Consistency checks

### 3.2.1 Duplicates

In [75]:
# check for duplicate values
dept.duplicated().sum()

0

There are no duplicates to clean.

### 3.2.2 Missing values

In [77]:
# check for nulls
dept.isnull().sum()

department_id
department    0
dtype: int64

There are no nulls to clean.

### 3.2.3 Inaccuracies

In [71]:
# view dataframe
dept

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


A scan of the department names suggests that some of the entries should be renamed:

+ 'dry goods pasta' to 'dry goods'
+ 'dairy eggs' to 'dairy'

and there is a 'missing' department.

Check with the client.

It is verified that the suggested name changes can be made. Still waiting on verification for the 'missing' department.

In [72]:
# Rename 'dry goods pasta' to 'dry goods' and 'dairy eggs' to 'dairy'
dept['department'] = dept['department'].replace({'dry goods pasta': 'dry goods', 'dairy eggs': 'dairy'})

# 4. Customers

In [83]:
# find dimensions of dataset
cust.shape

(206209, 10)

In [85]:
# view dataset
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 4.1 Wrangling

### 4.1.1 Datatypes

In [87]:
# check datatypes
cust.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

user_id can be converted to strings, since they represent names rather than quantifiable data.

In [88]:
# convert user_id to string
cust['user_id'] = cust['user_id'].astype(str)

There are no further mixed datatypes to clean.

### 4.1.2 Columns to drop

First name and surname columns are PII and should be considered for removal from the analysis.

Verification with the client confirms that these columns can be removed from the analysis.


In [92]:
# drop first name and surname columns
cust = cust.drop('Surnam', axis=1)
cust = cust.drop('First Name', axis=1)

### 4.1.3 Columns to rename

In [95]:
# list the columns
cust.columns

Index(['user_id', 'Gender', 'STATE', 'Age', 'date_joined', 'n_dependants',
       'fam_status', 'income'],
      dtype='object')

Rename columns to all lower case to be consistent with formatting across the script. Correct misspelling of 'n_dependants' to 'n_dependents'. Rename 'fam_statust' to 'family_status' to be more intuitive.

In [98]:
# rename columns as described above
cust.rename(columns={'Gender': 'sex'}, inplace=True)
cust.rename(columns={'STATE': 'state'}, inplace=True)
cust.rename(columns={'Age': 'age'}, inplace=True)
cust.rename(columns={'n_dependants': 'n_dependents'}, inplace=True)
cust.rename(columns={'fam_status': 'family_status'}, inplace=True)

## 4.2 Consistency checks

### 4.2.1 Duplicates

In [101]:
# check for duplicate values
cust.duplicated().sum()

0

There are no duplicate rows.

### 4.2.2 Missing values

In [103]:
# check for nulls
cust.isnull().sum()

user_id          0
gender           0
state            0
age              0
date_joined      0
n_dependents     0
family_status    0
income           0
dtype: int64

There are no nulls to fix.

### 4.2.3 Inaccuracies

In [104]:
# find summary statistics for dataset
cust.describe()

,age,n_dependents,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


The counts for each variable are correct and there are no suspicious summary statistics. Take the dataset to be accurate.

# Export cleaned dataframes

In [79]:
# export dataframes
ords.to_csv(os.path.join(path,'Prepared Data','orders_cleaned.csv'))
prods.to_csv(os.path.join(path,'Prepared Data','products_cleaned.csv'))
dept.to_csv(os.path.join(path,'Prepared Data','departments_cleaned.csv'))

In [108]:
cust.to_csv(os.path.join(path,'Prepared Data','customers_cleaned.csv'))